In [2]:
import re
import nltk
import stanza
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.corpus import brown

In [3]:
# sub-genres
fiction = ['adventure','fiction','mystery' , 'romance', 'science_fiction']
nonfiction = ['government','hobbies','learned','news', 'reviews'] 

# fileids
fiction_ids = [x for y in fiction for x in brown.fileids(categories=y)]
nonfiction_ids = [x for y in nonfiction for x in brown.fileids(categories=y)]

### Lexical Density

In [4]:
# def _lex_den(temp):
#     feat = Features(nlp_pipeline=nlp, text=temp['text'])
#     pos_counts_output = feat._pos_counts()
#     pos_counts_output['id'] = temp['id']
#     pos_counts_output['label'] = temp['label']
#     return pos_counts_output

In [5]:
lex_den_data = []
for index, fileid in tqdm(enumerate(fiction_ids+nonfiction_ids)):
    paras = brown.paras(fileids=fileid)
    
    label = 1 if fileid in fiction_ids else 0

    for j, p in enumerate(paras):
        if len(p) > 4 and len(p) < 7:
            text = ' '
            for sent in p: 
                text = text + ' '.join(sent)+' '
            text = text.strip()
            # pos_counts = _lex_den(text)
            temp = {}
            temp['id'] =f'{fileid}_para_{j}'
            temp['label'] = label
            temp['text'] = text
            lex_den_data.append(temp)


324it [00:01, 253.11it/s]


In [6]:
import time
from joblib import Parallel, delayed

In [7]:
# ## Parellilizing the computation
# start_time = time.perf_counter()
# lex_den_outputs = Parallel(n_jobs=4, prefer="threads")(delayed(_lex_den)(dic) for dic in lex_den_data)
# finish_time = time.perf_counter()
# print(f"Finished in {finish_time-start_time} seconds")

In [10]:
text = lex_den_data[0]['text']
text

'When , in late afternoon on the last day in June , he saw two people top the ridge to the south and walk toward the house , he quit work immediately and strode to his rifle . It could be some kind of trick Budd had thought up . No one walked in this country , least of all Ed Dow or Dutch Renfro or any of the rest of the Bar B crew . Morgan watched the two figures for a time , puzzled . When they were closer and he saw that one was a woman , he was more puzzled than ever .'

In [11]:
from src.extract_all_features import Features
nlp = stanza.Pipeline(lang='en', processors={'tokenize': 'spacy'})

2023-11-14 15:35:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-11-14 15:35:08 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | spacy               |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-11-14 15:35:08 INFO: Using device: cpu
2023-11-14 15:35:08 INFO: Loading: tokenize
2023-11-14 15:35:08 INFO: Loading: pos
2023-11-14 15:35:08 INFO: Loading: lemma
2023-11-14 15:35:08 INFO: Loading: constituency
2023-11-14 15:35:09 INFO: Loading: depparse
2023-11-14 15:35:09 INFO: Loading: sentiment
2023-11-14 15:35:10 INFO: Loading: ner
2023-11-14 15:3

In [12]:
feat = Features(nlp_pipeline=nlp, text=text)

In [13]:
feat._extract_features(choice='raw')

{'avg_sen_len': 21.4, 'std_sen_len': 9.951884243699782}

In [16]:
doc = nlp(text)
sents = []
for sent in doc.sentences:
    sents.append(sent)

In [22]:
[' '.join([w.text for w in s.words]).strip()  for s in doc.sentences]

['When , in late afternoon on the last day in June , he saw two people top the ridge to the south and walk toward the house , he quit work immediately and strode to his rifle .',
 'It could be some kind of trick Budd had thought up .',
 'No one walked in this country , least of all Ed Dow or Dutch Renfro or any of the rest of the Bar B crew .',
 'Morgan watched the two figures for a time , puzzled .',
 'When they were closer and he saw that one was a woman , he was more puzzled than ever .']